In [1]:
import torch
import torch.nn as nn
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from torch.utils.data import DataLoader
from src.model import LinearModel
from src.data import dataset
from src.runner import train, test
from src.viz import loss_visualize, acc_visualize

In [2]:
import wandb

sweep_config = {
    'method': 'grid'
    }

metric = {
    'name': 'val_acc',
    'goal': 'maximize'   
    }

sweep_config['metric'] = metric

parameters_dict = {
    'learning_rate': {
        'values': [0.001]
        },
    'hidden_size': {
        'values': [[128,64], [256,128], [256,256], [256,64], [512,256], [512,512], [1024,512],[1024,1024]]
        },
    'num_epochs': {
          'values': [1500]
        },
    'batch_size': {
          'values': [128]
        },
    }

sweep_config['parameters'] = parameters_dict

sweep_id = wandb.sweep(sweep_config, project="696ds_deepmind")

wandb.init(config=sweep_config,project="696ds_deepmind", entity="696ds_deepmind")

Create sweep with ID: dk76papc
Sweep URL: https://wandb.ai/696ds_deepmind/696ds_deepmind/sweeps/dk76papc


wandb: Currently logged in as: 696ds_deepmind (use `wandb login --relogin` to force relogin)


In [3]:
def run():
    wandb.init(config=sweep_config,project="696ds_deepmind", entity="696ds_deepmind")
    ######################### Hyper-parameters #########################
    config = wandb.config

    out_size = 1
    num_epochs = config.num_epochs #1500
    learning_rate = config.learning_rate #0.001
    hidden_size = config.hidden_size #[40,20]
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    batch_size = config.batch_size #128
    data_normal = pd.read_csv("data_normal.csv")
    num_clusters = data_normal.cluster.nunique()
    with_clusters = False

    if with_clusters:
        input_cols = ["x_1", "x_2", "cluster"]
        input_size = 3
    else:
        input_cols = ["x_1", "x_2"]
        input_size = 2

    X_train, X_test, y_train, y_test = train_test_split(np.array(data_normal[input_cols]), 
                           np.array(data_normal["y"]), test_size=0.3)


    trainset = dataset(torch.tensor(X_train,dtype=torch.float32).to(device), \
                        torch.tensor(y_train,dtype=torch.float32).to(device))
    testset = dataset(torch.tensor(X_test,dtype=torch.float32).to(device), \
                        torch.tensor(y_test,dtype=torch.float32).to(device))

    #DataLoader
    trainloader = DataLoader(trainset,batch_size=batch_size,shuffle=True)
    valloader = DataLoader(testset,batch_size=batch_size,shuffle=True)

    # model definition
    model = LinearModel(input_size, hidden_size, out_size, with_clusters = with_clusters, num_clusters = num_clusters).to(device)
    criterion = nn.BCEWithLogitsLoss()
    # criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    model, tr_loss, tr_acc, val_acc = train(model, trainloader, valloader, \
                            optimizer, num_epochs, criterion)
    loss_visualize(tr_loss, "Loss vs iteration")
    acc_visualize([tr_acc, val_acc], \
                    ["training accuracy", "validation accuracy"], \
                    "Accuracy vs epochs")
    best_val_acc = max(val_acc)
    wandb.log({"val_acc": best_val_acc})
    print("Val_acc",best_val_acc)
    # wandb.log({"best_trn_acc": tr_acc}) 

In [ ]:
wandb.agent(sweep_id, run)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: 57j6k6j1 with config:
wandb: 	batch_size: 128
wandb: 	hidden_size: [128, 64]
wandb: 	learning_rate: 0.001
wandb: 	num_epochs: 1500
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


/opt/nidhi/assignments/696/696_experiments/src/data.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.x = torch.tensor(x,dtype=torch.float32)
/opt/nidhi/assignments/696/696_experiments/src/data.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.y = torch.tensor(y,dtype=torch.float32)
  0%|                                                                                                                                                                             | 1/1500 [00:00<05:22,  4.65it/s]

Loss:  0.6951740305833142


  7%|███████████▌                                                                                                                                                               | 101/1500 [00:25<07:21,  3.17it/s]

Loss:  0.534170801892425


...

 67%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                                        | 1001/1500 [12:06<09:44,  1.17s/it]

Loss:  0.4466436803340912


 73%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                                             | 1101/1500 [14:07<08:19,  1.25s/it]

Loss:  0.4311435523659292


 76%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                         | 1134/1500 [14:46<07:08,  1.17s/it]